<a href="https://colab.research.google.com/github/Arnobzzz/seizureSense/blob/main/1--%3Estatistical.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Copy the zip file to Colab working directory
!cp "/content/drive/MyDrive/mount_adora_edf_processed.zip" .

# Verify it’s copied
!ls -lh mount_adora_edf_processed.zip


Mounted at /content/drive
-rw------- 1 root root 2.9G Jul  5 08:11 mount_adora_edf_processed.zip


In [ ]:
import zipfile
import os

zip_path = "mount_adora_edf_processed.zip"
extract_path = "adora_epochs"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

# Check how many .fif files were extracted
fif_files = [f for f in os.listdir(extract_path) if f.endswith('.fif')]
print(f"Extracted {len(fif_files)} .fif files")


Extracted 0 .fif files


In [ ]:
!pip install mne
!pip install antropy

In [ ]:
import os
import numpy as np
import pandas as pd
import mne
from scipy.stats import skew, kurtosis
from antropy import spectral_entropy, hjorth_params
from tqdm import tqdm

# ---- Feature function (your version with fixes) ----
def extract_features_from_epoch(epoch_data, sfreq):
    features = []
    for ch in epoch_data:
        if np.std(ch) < 1e-10:  # Almost constant signal
            features.extend([0, 0, 0, 0, 0, 0, 0, 0])  # mean, std, skew, kurt, entropy, hjorth3
        else:
            features.append(np.mean(ch))
            features.append(np.std(ch))
            try:
                features.append(skew(ch))
            except:
                features.append(0)
            try:
                features.append(kurtosis(ch))
            except:
                features.append(0)

            try:
                se = spectral_entropy(ch, sfreq)
                features.append(se if np.isfinite(se) else 0)
            except:
                features.append(0)

            try:
                h = hjorth_params(ch)
                features.extend([val if np.isfinite(val) else 0 for val in h])
            except:
                features.extend([0, 0, 0])
    return features

# ---- Paths ----
fif_dir = "/content/adora_epochs/sust"  # <-- adjust if different
fif_files = sorted([f for f in os.listdir(fif_dir) if f.endswith('.fif')])

# ---- Feature extraction ----
X, y = [], []

for file in tqdm(fif_files):
    path = os.path.join(fif_dir, file)
    try:
        epochs = mne.read_epochs(path, preload=True, verbose='ERROR')
        sfreq = epochs.info['sfreq']

        # Label inference
        fname = file.lower()
        if fname.startswith(("fn", "mn")):
            label = 0
        elif fname.startswith(("fp", "mp")):
            label = 1
        else:
            continue  # skip unrecognized

        # Feature per epoch
        for epoch in epochs.get_data():
            feats = extract_features_from_epoch(epoch, sfreq)
            X.append(feats)
            y.append(label)

    except Exception as e:
        print(f"⚠️ Failed on {file}: {e}")
        continue

# ---- Save as DataFrame ----
df = pd.DataFrame(X)
df['label'] = y

# Clean up NaNs/infs if any
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df.dropna(inplace=True)

# ✅ Preview and Save
print("✅ Feature extraction complete. Sample:")
print(df.head())

# Optional: Save to CSV
# df.to_csv("adora_stat_features.csv", index=False)


100%|██████████| 210/210 [2:37:57<00:00, 45.13s/it]


✅ Feature extraction complete. Sample:
         0    1    2    3    4    5    6    7    8    9  ...  167  168  169  \
74315  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
74316  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
74317  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
74318  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
74319  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   

       170  171  172  173  174  175  label  
74315  0.0  0.0  0.0  0.0  0.0  0.0      1  
74316  0.0  0.0  0.0  0.0  0.0  0.0      1  
74317  0.0  0.0  0.0  0.0  0.0  0.0      1  
74318  0.0  0.0  0.0  0.0  0.0  0.0      1  
74319  0.0  0.0  0.0  0.0  0.0  0.0      1  

[5 rows x 177 columns]


In [ ]:
import os
import mne
import numpy as np

# Path to .fif files
fif_dir = "/content/adora_epochs/sust"
fif_files = sorted([f for f in os.listdir(fif_dir) if f.endswith('.fif')])

# Pick the first valid file (can be fn, fp, mp, mn — any is fine)
for file in fif_files:
    if file.lower().startswith(("fn", "fp", "mn", "mp")):
        chosen_file = file
        break

# Load one .fif file
file_path = os.path.join(fif_dir, chosen_file)
print(f"🔍 Reading file: {chosen_file}")
epochs = mne.read_epochs(file_path, preload=True, verbose='ERROR')

# Get sampling frequency
sfreq = epochs.info['sfreq']
print(f"Sampling freq: {sfreq}")

# Read first epoch
data = epochs.get_data()
epoch = data[0]  # shape should be (n_channels, n_times)

# Print debug info
print("Epoch shape (channels, time):", epoch.shape)
print("First 10 values of channel 0:", epoch[0][:10])
print("Mean of channel 0:", np.mean(epoch[0]))
print("Std of channel 0:", np.std(epoch[0]))
print("Below std threshold (< 1e-10)?", np.std(epoch[0]) < 1e-10)


🔍 Reading file: fn1-epo.fif
Sampling freq: 128.0
Epoch shape (channels, time): (21, 256)
First 10 values of channel 0: [1.02119968e-09 1.28634601e-05 1.86144043e-05 1.65071342e-05
 1.09433431e-05 4.55464715e-06 2.92918457e-06 8.96387792e-06
 1.49794114e-05 1.33477342e-05]
Mean of channel 0: -5.897725360642578e-08
Std of channel 0: 1.020383599903415e-05
Below std threshold (< 1e-10)? False


In [ ]:
import os
import numpy as np
import pandas as pd
import mne
from scipy.stats import skew, kurtosis
from antropy import spectral_entropy, hjorth_params
from tqdm import tqdm

# ---- Feature Extraction Function ----
def extract_features_from_epoch(epoch_data, sfreq):
    features = []
    for ch in epoch_data:
        # Basic statistics
        features.append(np.mean(ch))
        features.append(np.std(ch))
        try:
            features.append(skew(ch))
        except:
            features.append(0)
        try:
            features.append(kurtosis(ch))
        except:
            features.append(0)

        # Spectral entropy
        try:
            se = spectral_entropy(ch, sfreq)
            features.append(se if np.isfinite(se) else 0)
        except:
            features.append(0)

        # Hjorth parameters
        try:
            h = hjorth_params(ch)
            features.extend([val if np.isfinite(val) else 0 for val in h])
        except:
            features.extend([0, 0, 0])
    return features

# ---- Paths ----
fif_dir = "/content/adora_epochs/sust"
fif_files = sorted([f for f in os.listdir(fif_dir) if f.endswith('.fif')])

# ---- Feature Extraction ----
X, y = [], []

for file in tqdm(fif_files):
    path = os.path.join(fif_dir, file)
    try:
        epochs = mne.read_epochs(path, preload=True, verbose='ERROR')
        sfreq = epochs.info['sfreq']

        # Label inference
        fname = file.lower()
        if fname.startswith(("fn", "mn")):
            label = 0  # normal
        elif fname.startswith(("fp", "mp")):
            label = 1  # epileptic
        else:
            continue  # skip unrecognized

        for epoch in epochs.get_data():  # shape: (n_epochs, n_channels, n_times)
            feats = extract_features_from_epoch(epoch, sfreq)
            X.append(feats)
            y.append(label)

    except Exception as e:
        print(f"⚠️ Failed on {file}: {e}")
        continue

# ---- Save as DataFrame ----
df = pd.DataFrame(X)
df['label'] = y

# Clean up invalid values
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df.dropna(inplace=True)

# ✅ Done
print("✅ Feature extraction complete. Shape:", df.shape)
print(df.head())


  0%|          | 0/210 [00:00<?, ?it/s]/tmp/ipython-input-17-4063888179.py:17: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  features.append(skew(ch))
/tmp/ipython-input-17-4063888179.py:21: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  features.append(kurtosis(ch))
/usr/local/lib/python3.11/dist-packages/antropy/entropy.py:255: RuntimeWarning: invalid value encountered in divide
  psd_norm = psd / psd.sum(axis=axis, keepdims=True)
/usr/local/lib/python3.11/dist-packages/antropy/entropy.py:1033: RuntimeWarning: invalid value encountered in scalar divide
  mob = np.sqrt(dx_var / x_var)
/usr/local/lib/python3.11/dist-packages/antropy/entropy.py:1034: RuntimeWarning: invalid value encountered in scalar divide
  com = np.sqrt(ddx_var / dx_v

✅ Feature extraction complete. Shape: (2858, 155)
                  0         1         2         3         4         5  \
74304  8.441343e-06  0.000022  0.009405 -0.745046  2.704163  0.093970   
74305 -1.479218e-06  0.000021 -0.180195 -0.751174  2.636994  0.106858   
74306 -5.644366e-07  0.000022 -0.292148 -0.916906  1.333779  0.083881   
74307  8.791004e-07  0.000021 -0.200704 -1.135979  1.004301  0.072796   
74308 -1.694445e-06  0.000018  0.279940 -1.054940  2.434396  0.111313   

              6             7         8         9  ...       145       146  \
74304  5.057449  7.285037e-06  0.000018  0.251147  ...  0.148131  4.773453   
74305  4.738832 -9.719888e-07  0.000016  0.291270  ...  0.135395  4.812585   
74306  6.486255 -1.699815e-06  0.000016  0.504585  ...  0.144550  4.678417   
74307  8.482261  5.752980e-07  0.000015  0.484156  ...  0.164294  5.233355   
74308  6.194054 -3.169167e-08  0.000023 -1.045828  ...  0.157713  4.211348   

            147       148       149       

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
import numpy as np

# 1. Prepare the data
X = df.drop(columns='label').values
y = df['label'].values

# 2. Split into training and test sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# 3. Initialize and train Random Forest
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# 4. Predict on test set
y_pred = clf.predict(X_test)

# 5. Evaluate
acc = accuracy_score(y_test, y_pred)
print(f"✅ Accuracy: {acc:.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred))


✅ Accuracy: 1.0000

Classification Report:
              precision    recall  f1-score   support

           1       1.00      1.00      1.00       572

    accuracy                           1.00       572
   macro avg       1.00      1.00      1.00       572
weighted avg       1.00      1.00      1.00       572



In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score

# 1. Prepare the data
X = df.drop(columns='label').values
y = df['label'].values

# 2. Train-test split (same as before)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# 3. Shuffle the labels to break any real patterns
y_train_shuffled = shuffle(y_train, random_state=42)

# 4. Train on shuffled labels
clf_shuffled = RandomForestClassifier(n_estimators=100, random_state=42)
clf_shuffled.fit(X_train, y_train_shuffled)

# 5. Predict on real test set
y_pred_shuffled = clf_shuffled.predict(X_test)

# 6. Evaluate accuracy
acc_shuffled = accuracy_score(y_test, y_pred_shuffled)
print(f"🧪 Accuracy with shuffled labels: {acc_shuffled:.4f} (expected ~0.5 if no leakage)")


🧪 Accuracy with shuffled labels: 1.0000 (expected ~0.5 if no leakage)


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold
import numpy as np

# Prepare features and labels
X = df.drop(columns='label').values
y = df['label'].values

# Define model
clf = RandomForestClassifier(n_estimators=100, random_state=42)

# Define stratified K-fold cross-validator
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Run cross-validation
scores = cross_val_score(clf, X, y, cv=cv)

# Report results
print("✅ Cross-Validation Accuracy Scores:", scores)
print(f"📊 Mean Accuracy: {np.mean(scores):.4f} ± {np.std(scores):.4f}")


✅ Cross-Validation Accuracy Scores: [1. 1. 1. 1. 1.]
📊 Mean Accuracy: 1.0000 ± 0.0000


In [ ]:
print(df.describe())


                  0             1            2            3            4  \
count  2.858000e+03  2.858000e+03  2858.000000  2858.000000  2858.000000   
mean  -7.048051e-09  8.830520e-06    -0.055977     0.292886     3.930889   
std    1.673161e-06  1.649194e-05     0.574572     1.567440     0.858260   
min   -2.516617e-05  2.055859e-08    -4.507024    -1.467988     0.991689   
25%   -3.482929e-07  1.910768e-06    -0.300579    -0.405232     3.217736   
50%   -4.917426e-09  6.954951e-06    -0.030890    -0.058615     4.087173   
75%    3.227941e-07  9.587578e-06     0.214360     0.438118     4.634098   
max    1.990958e-05  2.707606e-04     3.696379    27.231603     5.755339   

                 5            6             7             8            9  ...  \
count  2858.000000  2858.000000  2.858000e+03  2.858000e+03  2858.000000  ...   
mean      0.363788     3.240848  6.806272e-09  2.313434e-05    -0.014758  ...   
std       0.176910     1.553975  8.646824e-06  8.077701e-05     0.662513

In [ ]:
print("Sample filenames:")
print(fif_files[:100])

Sample filenames:
['fn1-epo.fif', 'fn10-epo.fif', 'fn11-epo.fif', 'fn12-epo.fif', 'fn13-epo.fif', 'fn14-epo.fif', 'fn15-epo.fif', 'fn16-epo.fif', 'fn17-epo.fif', 'fn18-epo.fif', 'fn19-epo.fif', 'fn2-epo.fif', 'fn20-epo.fif', 'fn21-epo.fif', 'fn22-epo.fif', 'fn23-epo.fif', 'fn24-epo.fif', 'fn25-epo.fif', 'fn26-epo.fif', 'fn27-epo.fif', 'fn28-epo.fif', 'fn29-epo.fif', 'fn3-epo.fif', 'fn30-epo.fif', 'fn31-epo.fif', 'fn32-epo.fif', 'fn33-epo.fif', 'fn34-epo.fif', 'fn35-epo.fif', 'fn36-epo.fif', 'fn37-epo.fif', 'fn38-epo.fif', 'fn39-epo.fif', 'fn4-epo.fif', 'fn40-epo.fif', 'fn41-epo.fif', 'fn42-epo.fif', 'fn43-epo.fif', 'fn44-epo.fif', 'fn45-epo.fif', 'fn46-epo.fif', 'fn47-epo.fif', 'fn48-epo.fif', 'fn49-epo.fif', 'fn5-epo.fif', 'fn50-epo.fif', 'fn51-epo.fif', 'fn6-epo.fif', 'fn7-epo.fif', 'fn8-epo.fif', 'fn9-epo.fif', 'fp1-epo.fif', 'fp10-epo.fif', 'fp11-epo.fif', 'fp12-epo.fif', 'fp13-epo.fif', 'fp14-epo.fif', 'fp15-epo.fif', 'fp16-epo.fif', 'fp17-epo.fif', 'fp18-epo.fif', 'fp19-epo.fif',

In [ ]:
from collections import Counter

# Make sure you include this after your df is created
label_counts = Counter(df['label'])
print("✅ Label distribution:", label_counts)


✅ Label distribution: Counter({1: 2858})


In [ ]:
normal_count = 0
epileptic_count = 0
X, y = [], []

for file in tqdm(fif_files):
    path = os.path.join(fif_dir, file)
    try:
        epochs = mne.read_epochs(path, preload=True, verbose='ERROR')
        sfreq = epochs.info['sfreq']
        fname = file.lower()

        if fname.startswith(("fn", "mn")):
            label = 0
            normal_count += len(epochs)
            print(f"✅ Processed NORMAL: {file} ({len(epochs)} epochs)")
        elif fname.startswith(("fp", "mp")):
            label = 1
            epileptic_count += len(epochs)
            print(f"✅ Processed EPILEPTIC: {file} ({len(epochs)} epochs)")
        else:
            print(f"⚠️ Skipping: {file}")
            continue

        for epoch in epochs.get_data():
            feats = extract_features_from_epoch(epoch, sfreq)
            X.append(feats)
            y.append(label)

    except Exception as e:
        print(f"❌ Failed to process {file}: {e}")
        continue

print(f"\n📊 Normal epochs: {normal_count}")
print(f"📊 Epileptic epochs: {epileptic_count}")


  0%|          | 0/210 [00:00<?, ?it/s]

✅ Processed NORMAL: fn1-epo.fif (1207 epochs)


/tmp/ipython-input-17-4063888179.py:17: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  features.append(skew(ch))
/tmp/ipython-input-17-4063888179.py:21: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  features.append(kurtosis(ch))
/usr/local/lib/python3.11/dist-packages/antropy/entropy.py:255: RuntimeWarning: invalid value encountered in divide
  psd_norm = psd / psd.sum(axis=axis, keepdims=True)
/usr/local/lib/python3.11/dist-packages/antropy/entropy.py:1033: RuntimeWarning: invalid value encountered in scalar divide
  mob = np.sqrt(dx_var / x_var)
/usr/local/lib/python3.11/dist-packages/antropy/entropy.py:1034: RuntimeWarning: invalid value encountered in scalar divide
  com = np.sqrt(ddx_var / dx_var) / mob
  0%|          | 1/210 [00:55

✅ Processed NORMAL: fn10-epo.fif (1205 epochs)


/tmp/ipython-input-17-4063888179.py:17: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  features.append(skew(ch))
/tmp/ipython-input-17-4063888179.py:21: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  features.append(kurtosis(ch))
/usr/local/lib/python3.11/dist-packages/antropy/entropy.py:255: RuntimeWarning: invalid value encountered in divide
  psd_norm = psd / psd.sum(axis=axis, keepdims=True)
/usr/local/lib/python3.11/dist-packages/antropy/entropy.py:1033: RuntimeWarning: invalid value encountered in scalar divide
  mob = np.sqrt(dx_var / x_var)
/usr/local/lib/python3.11/dist-packages/antropy/entropy.py:1034: RuntimeWarning: invalid value encountered in scalar divide
  com = np.sqrt(ddx_var / dx_var) / mob
  1%|          | 2/210 [01:47

...

✅ Processed EPILEPTIC: mp8-epo.fif (996 epochs)


/tmp/ipython-input-17-4063888179.py:17: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  features.append(skew(ch))
/tmp/ipython-input-17-4063888179.py:21: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  features.append(kurtosis(ch))
/usr/local/lib/python3.11/dist-packages/antropy/entropy.py:255: RuntimeWarning: invalid value encountered in divide
  psd_norm = psd / psd.sum(axis=axis, keepdims=True)
/usr/local/lib/python3.11/dist-packages/antropy/entropy.py:1033: RuntimeWarning: invalid value encountered in scalar divide
  mob = np.sqrt(dx_var / x_var)
/usr/local/lib/python3.11/dist-packages/antropy/entropy.py:1034: RuntimeWarning: invalid value encountered in scalar divide
  com = np.sqrt(ddx_var / dx_var) / mob
100%|█████████▉| 209/210 [2:3

✅ Processed EPILEPTIC: mp9-epo.fif (903 epochs)


/tmp/ipython-input-17-4063888179.py:17: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  features.append(skew(ch))
/tmp/ipython-input-17-4063888179.py:21: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  features.append(kurtosis(ch))
/usr/local/lib/python3.11/dist-packages/antropy/entropy.py:255: RuntimeWarning: invalid value encountered in divide
  psd_norm = psd / psd.sum(axis=axis, keepdims=True)
/usr/local/lib/python3.11/dist-packages/antropy/entropy.py:1033: RuntimeWarning: invalid value encountered in scalar divide
  mob = np.sqrt(dx_var / x_var)
/usr/local/lib/python3.11/dist-packages/antropy/entropy.py:1034: RuntimeWarning: invalid value encountered in scalar divide
  com = np.sqrt(ddx_var / dx_var) / mob
100%|██████████| 210/210 [2:3


📊 Normal epochs: 95114
📊 Epileptic epochs: 117844


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler

# Split features and labels
X_clean = df.drop('label', axis=1).values
y_clean = df['label'].values

# Normalize
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_clean)

# Define model
model = RandomForestClassifier(n_estimators=100, random_state=42)

# Cross-validation
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scores = cross_val_score(model, X_scaled, y_clean, cv=cv)

# Report
print(f"✅ Cross-Validation Accuracy Scores: {scores}")
print(f"📊 Mean Accuracy: {scores.mean():.4f} ± {scores.std():.4f}")


✅ Cross-Validation Accuracy Scores: [1. 1. 1. 1. 1.]
📊 Mean Accuracy: 1.0000 ± 0.0000


In [ ]:
import os
import numpy as np
import pandas as pd
import mne
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor

# ---- Lightweight Feature Extractor ----
def extract_features_from_epoch_fast(epoch_data):
    features = []
    for ch in epoch_data:
        features.append(np.mean(ch))
        features.append(np.std(ch))
        features.append(np.min(ch))
        features.append(np.max(ch))
    return features

# ---- Paths ----
fif_dir = "/content/adora_epochs/sust"
fif_files = sorted([f for f in os.listdir(fif_dir) if f.endswith('.fif')])
EPOCHS_PER_FILE_LIMIT = 50  # Limit epochs per file for speed

# ---- Process One File (Parallel-Friendly) ----
def process_file(file_idx, file):
    path = os.path.join(fif_dir, file)
    local_X, local_y, local_groups = [], [], []

    try:
        epochs = mne.read_epochs(path, preload=True, verbose='ERROR')
        sfreq = epochs.info['sfreq']
        fname = file.lower()

        if fname.startswith(("fn", "mn")):
            label = 0
        elif fname.startswith(("fp", "mp")):
            label = 1
        else:
            return [], [], []

        for epoch in epochs.get_data()[:EPOCHS_PER_FILE_LIMIT]:
            feats = extract_features_from_epoch_fast(epoch)
            if len(feats) > 0:
                local_X.append(feats)
                local_y.append(label)
                local_groups.append(file_idx)

        return local_X, local_y, local_groups

    except Exception:
        return [], [], []

# ---- Parallel Processing ----
X_all, y_all, groups_all = [], [], []
with ThreadPoolExecutor(max_workers=8) as executor:
    futures = [executor.submit(process_file, i, file) for i, file in enumerate(fif_files)]
    for f in tqdm(futures, desc="Extracting features"):
        Xf, yf, gf = f.result()
        X_all.extend(Xf)
        y_all.extend(yf)
        groups_all.extend(gf)

# ---- Save to CSV for Future Use ----
df = pd.DataFrame(X_all)
df['label'] = y_all
df['group'] = groups_all
df.to_csv("features_fast.csv", index=False)

df.shape


Extracting features: 100%|██████████| 210/210 [00:54<00:00,  3.87it/s]


(10500, 90)

In [ ]:
import os
import numpy as np
import pandas as pd
import mne
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor

# ---- Lightweight Feature Extractor ----
def extract_features_from_epoch_fast(epoch_data):
    features = []
    for ch in epoch_data:
        features.extend([
            np.mean(ch),
            np.std(ch),
            np.min(ch),
            np.max(ch)
        ])
    return features

# ---- Paths ----
fif_dir = "/content/adora_epochs/sust"
fif_files = sorted([f for f in os.listdir(fif_dir) if f.endswith('.fif')])
EPOCHS_PER_FILE_LIMIT = 50  # Speed-up: use max 50 epochs per file

# ---- Process One File (Parallel-Friendly) ----
def process_file(file_idx, file):
    path = os.path.join(fif_dir, file)
    local_X, local_y, local_groups = [], [], []

    try:
        epochs = mne.read_epochs(path, preload=True, verbose='ERROR')
        fname = file.lower()

        if fname.startswith(("fn", "mn")):
            label = 0  # Normal
        elif fname.startswith(("fp", "mp")):
            label = 1  # Epileptic
        else:
            return [], [], []

        # Limit to N epochs per file
        for epoch in epochs.get_data()[:EPOCHS_PER_FILE_LIMIT]:
            feats = extract_features_from_epoch_fast(epoch)
            local_X.append(feats)
            local_y.append(label)
            local_groups.append(file_idx)  # Use file index as group label

        return local_X, local_y, local_groups

    except Exception:
        return [], [], []

# ---- Parallel Feature Extraction ----
X_all, y_all, groups_all = [], [], []
with ThreadPoolExecutor(max_workers=8) as executor:
    futures = [executor.submit(process_file, i, file) for i, file in enumerate(fif_files)]
    for f in tqdm(futures, desc="Extracting features"):
        Xf, yf, gf = f.result()
        X_all.extend(Xf)
        y_all.extend(yf)
        groups_all.extend(gf)

# ---- Save to CSV ----
df = pd.DataFrame(X_all)
df['label'] = y_all
df['group'] = groups_all
df.to_csv("features_fast.csv", index=False)

print("✅ Saved to features_fast.csv")
print("📐 Data shape:", df.shape)


Extracting features: 100%|██████████| 210/210 [00:46<00:00,  4.51it/s]


✅ Saved to features_fast.csv
📐 Data shape: (10500, 90)


In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GroupKFold, cross_val_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

# ---- Load Features ----
df = pd.read_csv("features_fast.csv")

# ---- Prepare Data ----
X = df.drop(columns=["label", "group"]).values
y = df["label"].values
groups = df["group"].values

# ---- Classifier Pipeline ----
pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("clf", RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1))
])

# ---- Cross-Validation Setup (Group-wise to avoid leakage) ----
cv = GroupKFold(n_splits=5)
scores = cross_val_score(pipeline, X, y, cv=cv, groups=groups, scoring="accuracy")

# ---- Report ----
print(f"\n✅ Cross-Validation Accuracy Scores: {scores}")
print(f"📊 Mean Accuracy: {scores.mean():.4f} ± {scores.std():.4f}")

# ---- Train on Full Set & Evaluate on Same Set (optional) ----
pipeline.fit(X, y)
y_pred = pipeline.predict(X)

print("\n📈 Classification Report (on all data):")
print(classification_report(y, y_pred))

print("🧩 Confusion Matrix:")
print(confusion_matrix(y, y_pred))



✅ Cross-Validation Accuracy Scores: [0.64809524 0.61571429 0.65714286 0.51428571 0.59380952]
📊 Mean Accuracy: 0.6058 ± 0.0511

📈 Classification Report (on all data):
              precision    recall  f1-score   support

           0       0.99      0.98      0.99      4950
           1       0.98      0.99      0.99      5550

    accuracy                           0.99     10500
   macro avg       0.99      0.99      0.99     10500
weighted avg       0.99      0.99      0.99     10500

🧩 Confusion Matrix:
[[4844  106]
 [  34 5516]]


In [ ]:
# Re-import necessary libraries after code execution state reset
import os
import numpy as np
import pandas as pd
import mne
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor
from scipy.stats import skew, kurtosis
from antropy import spectral_entropy, hjorth_params

# ---- Richer Feature Extractor (Medium Complexity) ----
def extract_features_richer(epoch_data, sfreq):
    features = []
    for ch in epoch_data:
        features.append(np.mean(ch))
        features.append(np.std(ch))
        features.append(np.min(ch))
        features.append(np.max(ch))
        try:
            features.append(skew(ch))
        except:
            features.append(0)
        try:
            features.append(kurtosis(ch))
        except:
            features.append(0)
        try:
            features.append(spectral_entropy(ch, sfreq))
        except:
            features.append(0)
        try:
            features.extend(hjorth_params(ch))  # Activity, Mobility, Complexity
        except:
            features.extend([0, 0, 0])
    return features

# ---- Paths ----
fif_dir = "/content/adora_epochs/sust"
fif_files = sorted([f for f in os.listdir(fif_dir) if f.endswith('.fif')])
EPOCHS_PER_FILE_LIMIT = 50  # Limit epochs per file for speed

# ---- Process One File (Parallel-Friendly) ----
def process_file(file_idx, file):
    path = os.path.join(fif_dir, file)
    local_X, local_y, local_groups = [], [], []

    try:
        epochs = mne.read_epochs(path, preload=True, verbose='ERROR')
        sfreq = epochs.info['sfreq']
        fname = file.lower()

        if fname.startswith(("fn", "mn")):
            label = 0
        elif fname.startswith(("fp", "mp")):
            label = 1
        else:
            return [], [], []

        for epoch in epochs.get_data()[:EPOCHS_PER_FILE_LIMIT]:
            feats = extract_features_richer(epoch, sfreq)
            if len(feats) > 0:
                local_X.append(feats)
                local_y.append(label)
                local_groups.append(file_idx)

        return local_X, local_y, local_groups

    except Exception:
        return [], [], []

# ---- Parallel Processing ----
X_all, y_all, groups_all = [], [], []
with ThreadPoolExecutor(max_workers=8) as executor:
    futures = [executor.submit(process_file, i, file) for i, file in enumerate(fif_files)]
    for f in tqdm(futures, desc="Extracting features (richer)"):
        Xf, yf, gf = f.result()
        X_all.extend(Xf)
        y_all.extend(yf)
        groups_all.extend(gf)

# ---- Save to CSV for Future Use ----
df = pd.DataFrame(X_all)
df['label'] = y_all
df['group'] = groups_all
df.to_csv("features_medium_complexity.csv", index=False)

df.shape


Extracting features (richer):   0%|          | 0/210 [00:00<?, ?it/s]/tmp/ipython-input-8-194263269.py:20: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  features.append(skew(ch))
/tmp/ipython-input-8-194263269.py:24: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  features.append(kurtosis(ch))
/usr/local/lib/python3.11/dist-packages/antropy/entropy.py:255: RuntimeWarning: invalid value encountered in divide
  psd_norm = psd / psd.sum(axis=axis, keepdims=True)
/usr/local/lib/python3.11/dist-packages/antropy/entropy.py:1033: RuntimeWarning: invalid value encountered in scalar divide
  mob = np.sqrt(dx_var / x_var)
/usr/local/lib/python3.11/dist-packages/antropy/entropy.py:1034: RuntimeWarning: invalid value encountered in scalar divide
  co

(10500, 200)

In [ ]:
import os
import numpy as np
import pandas as pd
import mne
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor
from scipy.stats import skew, kurtosis
from antropy import spectral_entropy, hjorth_params

# ---- Feature Extractor (Medium Complexity) ----
def extract_features_medium(epoch_data, sfreq):
    features = []
    for ch in epoch_data:
        # Basic
        features.extend([np.mean(ch), np.std(ch), np.min(ch), np.max(ch)])

        # Shape features
        features.append(skew(ch))
        features.append(kurtosis(ch))

        # Hjorth
        activity, mobility, complexity = hjorth_params(ch)
        features.extend([activity, mobility, complexity])

        # Entropy
        features.append(spectral_entropy(ch, sfreq))

    return features

# ---- Paths ----
fif_dir = "/content/adora_epochs/sust"
fif_files = sorted([f for f in os.listdir(fif_dir) if f.endswith('.fif')])
EPOCHS_PER_FILE_LIMIT = 50

# ---- File Processor ----
def process_file(file_idx, file):
    path = os.path.join(fif_dir, file)
    local_X, local_y, local_groups = [], [], []

    try:
        epochs = mne.read_epochs(path, preload=True, verbose='ERROR')
        sfreq = epochs.info['sfreq']
        fname = file.lower()

        if fname.startswith(("fn", "mn")):
            label = 0
        elif fname.startswith(("fp", "mp")):
            label = 1
        else:
            return [], [], []

        for epoch in epochs.get_data()[:EPOCHS_PER_FILE_LIMIT]:
            feats = extract_features_medium(epoch, sfreq)
            if len(feats) > 0:
                local_X.append(feats)
                local_y.append(label)
                local_groups.append(file_idx)

        return local_X, local_y, local_groups

    except Exception:
        return [], [], []

# ---- Parallel Processing ----
X_all, y_all, groups_all = [], [], []
with ThreadPoolExecutor(max_workers=8) as executor:
    futures = [executor.submit(process_file, i, file) for i, file in enumerate(fif_files)]
    for f in tqdm(futures, desc="Extracting features (richer)"):
        Xf, yf, gf = f.result()
        X_all.extend(Xf)
        y_all.extend(yf)
        groups_all.extend(gf)

# ---- Save to CSV ----
df = pd.DataFrame(X_all)
df['label'] = y_all
df['group'] = groups_all
df.to_csv("features_richer.csv", index=False)

print("✅ Saved features_richer.csv")
print(df.shape)


Extracting features (richer):  45%|████▌     | 95/210 [00:16<00:31,  3.68it/s]/tmp/ipython-input-10-752732104.py:18: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  features.append(skew(ch))
/tmp/ipython-input-10-752732104.py:19: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  features.append(kurtosis(ch))
/usr/local/lib/python3.11/dist-packages/antropy/entropy.py:1033: RuntimeWarning: invalid value encountered in scalar divide
  mob = np.sqrt(dx_var / x_var)
/usr/local/lib/python3.11/dist-packages/antropy/entropy.py:1034: RuntimeWarning: invalid value encountered in scalar divide
  com = np.sqrt(ddx_var / dx_var) / mob
Extracting features (richer): 100%|██████████| 210/210 [00:33<00:00,  6.27it/s]

✅ Saved features_richer.csv
(0, 2)


#**Successful code**

In [ ]:
import os
import numpy as np
import pandas as pd
import mne
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor

# ---- Feature Extraction (Medium Complexity) ----
def extract_features_from_epoch_medium(epoch_data, sfreq):
    features = []
    for ch in epoch_data:
        features.append(np.mean(ch))
        features.append(np.std(ch))
        features.append(np.min(ch))
        features.append(np.max(ch))
        features.append(np.median(ch))
        features.append(np.percentile(ch, 25))
        features.append(np.percentile(ch, 75))
        features.append(np.sum(np.abs(np.diff(ch))))  # Signal mobility
        features.append(np.mean(np.abs(ch - np.mean(ch))))  # MAD
        features.append(np.max(np.abs(ch)))  # Peak
    return features

# ---- Setup ----
fif_dir = "/content/adora_epochs/sust"
fif_files = sorted([f for f in os.listdir(fif_dir) if f.endswith('.fif')])
EPOCHS_PER_FILE_LIMIT = 50  # You can raise to 100+ if needed

# ---- Process a File (Thread-Friendly) ----
def process_file(file_idx, file):
    path = os.path.join(fif_dir, file)
    local_X, local_y, local_groups = [], [], []

    try:
        epochs = mne.read_epochs(path, preload=True, verbose='ERROR')
        sfreq = epochs.info['sfreq']
        fname = file.lower()

        if fname.startswith(("fn", "mn")):
            label = 0
        elif fname.startswith(("fp", "mp")):
            label = 1
        else:
            return [], [], []

        for epoch in epochs.get_data()[:EPOCHS_PER_FILE_LIMIT]:
            feats = extract_features_from_epoch_medium(epoch, sfreq)
            if isinstance(feats, list) and all(isinstance(x, (float, int)) for x in feats):
                local_X.append(feats)
                local_y.append(label)
                local_groups.append(file_idx)

        return local_X, local_y, local_groups

    except Exception as e:
        print(f"❌ Error in file {file}: {e}")
        return [], [], []

# ---- Run in Parallel ----
X_all, y_all, groups_all = [], [], []
with ThreadPoolExecutor(max_workers=8) as executor:
    futures = [executor.submit(process_file, i, file) for i, file in enumerate(fif_files)]
    for f in tqdm(futures, desc="Extracting features (richer)"):
        Xf, yf, gf = f.result()
        X_all.extend(Xf)
        y_all.extend(yf)
        groups_all.extend(gf)

# ---- Ensure Consistent Length (Trim if needed) ----
if len(X_all) == 0:
    raise ValueError("❌ No feature vectors extracted. Check file names or data.")

min_len = min(len(x) for x in X_all)
X_all = [x[:min_len] for x in X_all]

X_np = np.array(X_all)
y_np = np.array(y_all)
groups_np = np.array(groups_all)

# ---- Save to CSV ----
df = pd.DataFrame(X_np)
df['label'] = y_np
df['group'] = groups_np
df.to_csv("features_medium.csv", index=False)

# ---- Final Shape ----
print("✅ Feature extraction complete!")
print("Shape of features:", X_np.shape)
print("Labels count:", np.bincount(y_np))


Extracting features (richer): 100%|██████████| 210/210 [02:47<00:00,  1.26it/s]


✅ Feature extraction complete!
Shape of features: (10500, 210)
Labels count: [4950 5550]


# **Randomforest**

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, cross_val_predict, StratifiedKFold
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

# ---- Use already loaded variables ----
X = X_np
y = y_np

# ---- Random Forest ----
clf = RandomForestClassifier(n_estimators=100, random_state=42)

# ---- Cross-Validation ----
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
cv_scores = cross_val_score(clf, X, y, cv=cv)
y_pred = cross_val_predict(clf, X, y, cv=cv)

# ---- Metrics ----
print(f"✅ Cross-Validation Accuracy Scores: {cv_scores}")
print(f"📊 Mean Accuracy: {cv_scores.mean():.4f} ± {cv_scores.std():.4f}\n")

print("📈 Classification Report (on all data):")
print(classification_report(y, y_pred))

print("🧩 Confusion Matrix:")
print(confusion_matrix(y, y_pred))


✅ Cross-Validation Accuracy Scores: [0.93238095 0.93714286 0.93047619 0.93333333 0.93095238]
📊 Mean Accuracy: 0.9329 ± 0.0024

📈 Classification Report (on all data):
              precision    recall  f1-score   support

           0       0.93      0.93      0.93      4950
           1       0.94      0.94      0.94      5550

    accuracy                           0.93     10500
   macro avg       0.93      0.93      0.93     10500
weighted avg       0.93      0.93      0.93     10500

🧩 Confusion Matrix:
[[4596  354]
 [ 351 5199]]


# **XGBoost**

In [ ]:
pip install xgboost


In [ ]:
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_score, cross_val_predict
from sklearn.metrics import classification_report, confusion_matrix

# ---- Load from CSV if needed ----
df = pd.read_csv("features_medium.csv")
X = df.drop(columns=["label", "group"]).values
y = df["label"].values

# ---- XGBoost Classifier ----
clf = XGBClassifier(
    n_estimators=100,
    max_depth=5,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    use_label_encoder=False,
    eval_metric='logloss',
    random_state=42
)

# ---- Cross-Validation ----
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
cv_scores = cross_val_score(clf, X, y, cv=cv)
y_pred = cross_val_predict(clf, X, y, cv=cv)

# ---- Evaluation ----
print(f"✅ CV Accuracy Scores: {cv_scores}")
print(f"📊 Mean Accuracy: {cv_scores.mean():.4f} ± {cv_scores.std():.4f}\n")

print("📈 Classification Report:")
print(classification_report(y, y_pred))

print("🧩 Confusion Matrix:")
print(confusion_matrix(y, y_pred))


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [19:26:51] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [19:27:01] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [19:27:05] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [19:27:08] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [19:27:18] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_e

✅ CV Accuracy Scores: [0.91809524 0.91904762 0.91619048 0.90333333 0.9152381 ]
📊 Mean Accuracy: 0.9144 ± 0.0057

📈 Classification Report:
              precision    recall  f1-score   support

           0       0.90      0.92      0.91      4950
           1       0.93      0.91      0.92      5550

    accuracy                           0.91     10500
   macro avg       0.91      0.91      0.91     10500
weighted avg       0.91      0.91      0.91     10500

🧩 Confusion Matrix:
[[4576  374]
 [ 525 5025]]


In [ ]:
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold, cross_val_score, cross_val_predict
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

# ---- Load CSV Features ----
df = pd.read_csv("features_medium.csv")
X = df.drop(columns=["label", "group"]).values
y = df["label"].values

# ---- SVM Classifier with Pipeline (StandardScaler + SVM) ----
clf = make_pipeline(
    StandardScaler(),
    SVC(kernel='rbf', C=1.0, gamma='scale', random_state=42)
)

# ---- Cross-Validation ----
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
cv_scores = cross_val_score(clf, X, y, cv=cv)
y_pred = cross_val_predict(clf, X, y, cv=cv)

# ---- Evaluation ----
print(f"✅ CV Accuracy Scores: {cv_scores}")
print(f"📊 Mean Accuracy: {cv_scores.mean():.4f} ± {cv_scores.std():.4f}\n")

print("📈 Classification Report:")
print(classification_report(y, y_pred))

print("🧩 Confusion Matrix:")
print(confusion_matrix(y, y_pred))


✅ CV Accuracy Scores: [0.66904762 0.66571429 0.67047619 0.63904762 0.68857143]
📊 Mean Accuracy: 0.6666 ± 0.0159

📈 Classification Report:
              precision    recall  f1-score   support

           0       0.69      0.54      0.60      4950
           1       0.65      0.78      0.71      5550

    accuracy                           0.67     10500
   macro avg       0.67      0.66      0.66     10500
weighted avg       0.67      0.67      0.66     10500

🧩 Confusion Matrix:
[[2661 2289]
 [1212 4338]]


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold, cross_val_score, cross_val_predict
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier

# ---- Load Features ----
df = pd.read_csv("features_medium.csv")
X = df.drop(['label', 'group'], axis=1).values
y = df['label'].values

# ---- Feature Scaling (Important for KNN) ----
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# ---- KNN Classifier ----
knn = KNeighborsClassifier(n_neighbors=5, metric='euclidean')

# ---- Cross-Validation ----
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scores = cross_val_score(knn, X_scaled, y, cv=cv)
y_pred = cross_val_predict(knn, X_scaled, y, cv=cv)

# ---- Evaluation ----
print(f"✅ Cross-Validation Accuracy Scores: {np.round(scores, 4)}")
print(f"📊 Mean Accuracy: {scores.mean():.4f} ± {scores.std():.4f}\n")

print("📈 Classification Report (on all data):")
print(classification_report(y, y_pred))

print("🧩 Confusion Matrix:")
print(confusion_matrix(y, y_pred))


✅ Cross-Validation Accuracy Scores: [0.8862 0.8919 0.8781 0.8724 0.8871]
📊 Mean Accuracy: 0.8831 ± 0.0070

📈 Classification Report (on all data):
              precision    recall  f1-score   support

           0       0.87      0.88      0.88      4950
           1       0.89      0.89      0.89      5550

    accuracy                           0.88     10500
   macro avg       0.88      0.88      0.88     10500
weighted avg       0.88      0.88      0.88     10500

🧩 Confusion Matrix:
[[4344  606]
 [ 621 4929]]


fghjkyyyyyyyyyyyyyyyyyyuyuyuytrrtrtjjkkkkkkjkhjhjhjjjhhggggghnjnjnjnjnnnjijijghhhgghgnjnmmnmnmmmnnhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhh